In [ ]:
import pandas as pd

In [ ]:
path = "/content/drive/MyDrive/"
cols = ['id', 'text', 'label', 'intensity']
anger = pd.read_csv(path + 'train_anger.txt', header=None, sep='\t', names=cols, index_col=0)
fear = pd.read_csv(path + 'train_fear.txt', header=None, sep='\t', names=cols, index_col=0)
sad = pd.read_csv(path + 'train_sadness.txt', header=None, sep='\t', names=cols, index_col=0)
joy = pd.read_csv(path + 'train_joy.txt', header=None, sep='\t', names=cols, index_col=0)

joy.head()

,text,label,intensity
id,,,
30000,Just got back from seeing @GaryDelaney in Burs...,joy,0.980
30001,Oh dear an evening of absolute hilarity I don'...,joy,0.958
30002,Been waiting all week for this game â¤ï¸â¤ï...,joy,0.940
30003,"@gardiner_love : Thank you so much, Gloria! Yo...",joy,0.938
30004,I feel so blessed to work with the family that...,joy,0.938


In [ ]:
express = [anger, fear, sad, joy]
df_train = pd.concat(express)
df_train.reset_index(inplace=True)
df_train.label.value_counts()

fear       1147
anger       857
joy         823
sadness     786
Name: label, dtype: int64

In [ ]:
df_train.tail()

,id,text,label,intensity
342,30897,It feels good to get outside for a minute and ...,joy,0.580
343,30898,@r0Ils ppl get triggered over u smiling they'r...,joy,0.170
344,30899,@GigaFag @pipertownsend_ snapchat new would be...,joy,0.396
345,30900,@GigaFag @pipertownsend_ snapchat new would be...,joy,0.156
346,30901,A hearty Jonza! to all my friends and follower.,joy,0.704


In [ ]:
# import seaborn as sns
# colors = ['red', 'black', 'lightblue', 'yellow']
# bplot = sns.boxplot(data=df_train, x='label', y='intensity')
# for i in range(4):
#     bplot.artists[i].set_facecolor(colors[i])
# plt.title('Average Intensity for Each Label')



In [ ]:
from nltk.corpus import stopwords

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
import string

In [ ]:
from nltk.tokenize import word_tokenize
import re
from nltk.stem import WordNetLemmatizer

In [ ]:
punctuations = string.punctuation
stopwords = stopwords.words("english")
total_stopwords = stopwords+list(punctuations)
total_stopwords.remove("not")

In [ ]:

def contractions(s):
 s = re.sub(r"won't", "will not",s)
 s = re.sub(r"would't", "would not",s)
 s = re.sub(r"could't", "could not",s)
 s = re.sub(r"\'d", "would",s)
 s = re.sub(r"can\'t", "can not",s)
 s = re.sub(r"n\'t", "not", s)
 s= re.sub(r"\'re", "are", s)
 s = re.sub(r"\'s", "is", s)
 s = re.sub(r"\"ll", "will", s)
 s = re.sub(r"\'t", "not", s)
 s = re.sub(r"\'ve", "have", s)
 s = re.sub(r"\'m", "am", s)
 s = re.sub("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"," ",s)
 s = re.sub("[^A-Za-z ]"," ",s)
 return s
df_train['text']=df_train['text'].apply(lambda x:contractions(x))

In [ ]:
lema = WordNetLemmatizer()
def clean_data(text):
    cleaned = []
    for headline in text:
        headlines = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ",headline)   #url
        headlines = re.sub("([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"," ",headlines)       #email
        headlines = re.sub("[^A-Za-z ]"," ",headlines)
        headlines = headlines.lower()
        headlines = word_tokenize(headlines)
        print(headlines)
        headlines = [lema.lemmatize(word) for word in headlines if word not in total_stopwords]
        cleaned.append(" ".join(headlines))
    return cleaned

In [ ]:
text = clean_data(df_train['text'])

In [ ]:
df_train.text = pd.DataFrame(text)
df_train.head()

,id,text,label,intensity
0,10000,fu k heck moved fridge knock landlord door ang...,anger,0.938
1,10001,indian uber driver called someone n word wasno...,anger,0.896
2,10002,dpd uk asked parcel delivered pick store not a...,anger,0.896
3,10003,ef whichever butt wipe pulled fire alarm davis...,anger,0.896
4,10004,donot join btcare put phone talk rude taking m...,anger,0.896


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
label_encoder = LabelEncoder()

In [ ]:
df_train['label']= label_encoder.fit_transform(df_train['label'])

In [ ]:
df_train.tail()

,id,text,label,intensity
3608,30818,itis lack company liveliness make bored,2,0.058
3609,30819,quinnis short hair make sad glee,2,0.040
3610,30820,hate overthinking e v e r h n g like jus wan n...,2,0.040
3611,30821,people cheer sport team completely outside new...,2,0.020
3612,30822,damnpatriot youare po rejoicing someoneis death,2,0.019


In [ ]:
x_df_train = df_train.drop('intensity',axis=1)
x_df_train.shape

(3613, 3)

In [ ]:
y_df_train = df_train['intensity']
y_df_train.shape

(3613,)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vectorizer_tfidf = TfidfVectorizer(max_features=1000)
x_tfidf = vectorizer_tfidf.fit_transform(x_df_train['text'])
x_tfidf = pd.DataFrame.sparse.from_spmatrix(x_tfidf).join(x_df_train['label'])

In [ ]:
x_tfidf

,0,1,2,3,4,5,6,7,8,9,...,991,992,993,994,995,996,997,998,999,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.407917,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3608,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2
3609,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2
3610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2
3611,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,2


In [ ]:
x_tfidf.columns = x_tfidf.columns.astype(str)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import svm

In [ ]:
#linear regression
tfidf_linear = LinearRegression().fit(x_tfidf, y_df_train)
#svm regressor
tfidf_svr = svm.SVR().fit(x_tfidf, y_df_train)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [ ]:
path = "/content/drive/MyDrive/"
cols = ['id', 'text', 'label', 'intensity']
dev_anger = pd.read_csv(path + 'dev_anger.txt', header=None, sep='\t', names=cols, index_col=0)
dev_fear = pd.read_csv(path + 'dev_fear.txt', header=None, sep='\t', names=cols, index_col=0)
dev_sad = pd.read_csv(path + 'dev_sadness.txt', header=None, sep='\t', names=cols, index_col=0)
dev_joy = pd.read_csv(path + 'dev_joy.txt', header=None, sep='\t', names=cols, index_col=0)
dev_anger_intensity = pd.read_csv(path + 'dev_anger_intensity.txt', header=None, sep='\t', names=cols, index_col=0)
dev_fear_intensity = pd.read_csv(path + 'dev_fear_intensity.txt', header=None, sep='\t', names=cols, index_col=0)
dev_sad_intensity = pd.read_csv(path + 'dev_sadness_intensity.txt', header=None, sep='\t', names=cols, index_col=0)
dev_joy_intensity = pd.read_csv(path + 'dev_joy_intensity.txt', header=None, sep='\t', names=cols, index_col=0)

dev_joy.head()

,text,label,intensity
id,,,
30823,"@theclobra lol I thought maybe, couldn't decid...",joy,NONE
30824,Nawaz Sharif is getting more funnier than @kap...,joy,NONE
30825,Nawaz Sharif is getting more funnier than @kap...,joy,NONE
30826,@tomderivan73 ðŸ˜...I'll just people watch an...,joy,NONE
30827,I love my family so much #lucky #grateful #sma...,joy,NONE


In [ ]:
express = [dev_anger, dev_fear, dev_sad, dev_joy,dev_anger_intensity, dev_fear_intensity, dev_sad_intensity, dev_joy_intensity]
dev_df = pd.concat(express)
dev_df.reset_index(inplace=True)
dev_df.label.value_counts()

anger      844
fear       220
joy        158
sadness    148
Name: label, dtype: int64

In [ ]:
dev_df['text']=dev_df['text'].apply(lambda x:contractions(x))

In [ ]:
text = clean_data(dev_df['text'])

['at', 'the', 'point', 'today', 'where', 'if', 'someone', 'says', 'something', 'remotely', 'kind', 'to', 'me', 'a', 'waterfall', 'will', 'burst', 'out', 'of', 'my', 'eyes']
['corningfootball', 'it', 's', 'game', 'day', 't', 'minus', 'relentless']
['this', 'game', 'has', 'pissed', 'me', 'off', 'more', 'than', 'any', 'other', 'game', 'this', 'year', 'my', 'blood', 'is', 'boiling', 'time', 'to', 'turn', 'it', 'off', 'stlcards']
['spamvicious', 'ihave', 'just', 'found', 'out', 'itis', 'candice', 'and', 'not', 'candace', 'she', 'can', 'pout', 'all', 'she', 'likes', 'for', 'me']
['moocowward', 'mrsajhargreaves', 'melly', 'garybarlow', 'if', 'he', 'can', 'not', 'come', 'to', 'my', 'mum', 'a', 'th', 'after', 'k', 'tweets', 'then', 'why', 'should', 'i', 'soreloser']
['moocowward', 'mrsajhargreaves', 'melly', 'garybarlow', 'if', 'he', 'can', 'not', 'come', 'to', 'my', 'mum', 'a', 'th', 'after', 'k', 'tweets', 'then', 'why', 'should', 'i', 'bitter', 'soreloser']
['wan', 'na', 'go', 'home', 'and',

In [ ]:
dev_df.text = pd.DataFrame(text)

In [ ]:
dev_df['label']= label_encoder.fit_transform(dev_df['label'])

In [ ]:
x_dev_df = dev_df.drop('intensity',axis=1)
x_dev_df.shape

(1370, 3)

In [ ]:
x_dev_df

,id,text,label
0,10941,point today someone say something remotely kin...,0
1,10942,corningfootball game day minus relentless,0
2,10943,game pissed game year blood boiling time turn ...,0
3,10944,spamvicious ihave found itis candice not canda...,0
4,10945,moocowward mrsajhargreaves melly garybarlow no...,0
...,...,...,...
1365,30897,feel good get outside minute get fresh air iti...,2
1366,30898,r il ppl get triggered u smiling theyare irrel...,2
1367,30899,gigafag pipertownsend snapchat new would beg d...,2
1368,30900,gigafag pipertownsend snapchat new would beg d...,2


In [ ]:
y_dev_df = dev_df['intensity']
y_dev_df

0        NONE
1        NONE
2        NONE
3        NONE
4        NONE
        ...  
1365     0.58
1366     0.17
1367    0.396
1368    0.156
1369    0.704
Name: intensity, Length: 1370, dtype: object

In [ ]:
y_dev_df=pd.to_numeric(y_dev_df, errors='coerce').fillna(0)
y_dev_df

0       0.000
1       0.000
2       0.000
3       0.000
4       0.000
        ...  
1365    0.580
1366    0.170
1367    0.396
1368    0.156
1369    0.704
Name: intensity, Length: 1370, dtype: float64

In [ ]:
vectorizer_tfidf = TfidfVectorizer(max_features=1000)
x_dev_df_tfidf = vectorizer_tfidf.fit_transform(x_dev_df['text'])
x_dev_df_tfidf = pd.DataFrame.sparse.from_spmatrix(x_dev_df_tfidf).join(x_dev_df['label'])

In [ ]:
x_dev_df_tfidf.columns = x_dev_df_tfidf.columns.astype(str)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

In [ ]:
def get_score(y_test, y_pred, name):
    print(name + '\n')
    print('Mean Absolute Error:', mean_absolute_error(y_test, y_pred))
    print('Mean Squared Error:', mean_squared_error(y_test, y_pred))
    print('Root Mean Squared Error:', np.sqrt(mean_squared_error(y_test, y_pred)))
    return mean_absolute_error(y_test, y_pred),mean_squared_error(y_test, y_pred),np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
tfidf_linear.predict(x_dev_df_tfidf)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


array([0.39336391, 0.44017057, 0.34526427, ..., 0.79098406, 0.82631313,
       0.40591657])

In [ ]:
predict = {'linear':tfidf_linear.predict(x_dev_df_tfidf),'svm':tfidf_svr.predict(x_dev_df_tfidf)}
for i in predict:
  get_score(y_dev_df,predict[i],i)


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


linear

Mean Absolute Error: 0.39424495016043726
Mean Squared Error: 0.1934807086210772
Root Mean Squared Error: 0.4398644207265202
svm

Mean Absolute Error: 0.3915783784734369
Mean Squared Error: 0.18103795717282467
Root Mean Squared Error: 0.4254855545994772


In [ ]:
y_actual = y_dev_df
y_predicted = tfidf_svr.predict(x_dev_df_tfidf)

pd.DataFrame(data={"Actual Intensity" : y_actual, "Predicted Intensity" : y_predicted})

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:768: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


,Actual Intensity,Predicted Intensity
0,0.000,0.478089
1,0.000,0.365400
2,0.000,0.326366
3,0.000,0.399231
4,0.000,0.500648
...,...,...
1365,0.580,0.292582
1366,0.170,0.563261
1367,0.396,0.506788
1368,0.156,0.533769


In [ ]:
combined_df = pd.concat([df_train[['id', 'text', 'label', 'intensity']], dev_df]).reset_index()
combined_df.shape

(4983, 5)

In [ ]:
combined_df

,index,id,text,label,intensity
0,0,10000,fu k heck moved fridge knock landlord door ang...,0,0.938
1,1,10001,indian uber driver called someone n word wasno...,0,0.896
2,2,10002,dpd uk asked parcel delivered pick store not a...,0,0.896
3,3,10003,ef whichever butt wipe pulled fire alarm davis...,0,0.896
4,4,10004,donot join btcare put phone talk rude taking m...,0,0.896
...,...,...,...,...,...
4978,1365,30897,feel good get outside minute get fresh air iti...,2,0.58
4979,1366,30898,r il ppl get triggered u smiling theyare irrel...,2,0.17
4980,1367,30899,gigafag pipertownsend snapchat new would beg d...,2,0.396
4981,1368,30900,gigafag pipertownsend snapchat new would beg d...,2,0.156


In [ ]:
combined_df['intensity']=pd.to_numeric(combined_df['intensity'], errors='coerce').fillna(0)


In [ ]:
anger = combined_df.loc[combined_df['label']==0]
anger_vectorizer = TfidfVectorizer(max_features=1000)
x_anger = anger_vectorizer.fit_transform(anger['text'])
anger_model = svm.SVR().fit(x_anger, anger['intensity'])

#FEAR
fear = combined_df.loc[combined_df['label']==1]
fear_vectorizer = TfidfVectorizer(max_features=1000)
x_fear = fear_vectorizer.fit_transform(fear['text'])
fear_model = svm.SVR().fit(x_fear, fear['intensity'])

#SADNESS
sad = combined_df.loc[combined_df['label']==3]
sad_vectorizer = TfidfVectorizer(max_features=1000)
x_sad = sad_vectorizer.fit_transform(sad['text'])
sad_model = svm.SVR().fit(x_sad, sad['intensity'])

#JOY
joy = combined_df.loc[combined_df['label']==2]
joy_vectorizer = TfidfVectorizer(max_features=1000)
x_joy = joy_vectorizer.fit_transform(joy['text'])
joy_model = svm.SVR().fit(x_joy, joy['intensity'])

In [ ]:
cols = ["id", "text", "label", "intensity"]
anger_test = pd.read_csv(path + "test_anger.txt", header=None, sep="\t", names=cols, index_col=0)
fear_test = pd.read_csv(path + "test_fear.txt", header=None, sep="\t", names=cols, index_col=0)
sad_test = pd.read_csv(path + "test_sadness.txt", header=None, sep="\t", names=cols, index_col=0)
joy_test = pd.read_csv(path + "test_joy.txt", header=None, sep="\t", names=cols, index_col=0)
anger_test_intensity = pd.read_csv(path + "test_anger_intensity.txt", header=None, sep="\t", names=cols, index_col=0)
fear_test_intensity = pd.read_csv(path + "test_fear_intensity.txt", header=None, sep="\t", names=cols, index_col=0)
sad_test_intensity = pd.read_csv(path + "test_sadness_intensity.txt", header=None, sep="\t", names=cols, index_col=0)
joy_test_intensity = pd.read_csv(path + "test_joy_intensity.txt", header=None, sep="\t", names=cols, index_col=0)

anger_test_intensity.head()

,text,label,intensity
id,,,
10941,At the point today where if someone says somet...,anger,NONE
10942,@CorningFootball IT'S GAME DAY!!!! T MIN...,anger,NONE
10943,This game has pissed me off more than any othe...,anger,NONE
10944,@spamvicious I've just found out it's Candice ...,anger,NONE
10945,@moocowward @mrsajhargreaves @Melly77 @GaryBar...,anger,NONE


In [ ]:
w_intensity = [anger_test,fear_test,sad_test,joy_test,anger_test_intensity,fear_test_intensity,sad_test_intensity,joy_test_intensity]
for i in w_intensity:
  i['intensity']=pd.to_numeric(i['intensity'], errors='coerce').fillna(0)
  text = clean_data(i['text'])
  i['text']=i['text'].apply(lambda x:contractions(x))


In [ ]:
express = [anger_test,anger_test_intensity]
df_test_anger = pd.concat(express)
df_test_anger.reset_index(inplace=True)
df_test_anger.label.value_counts()

anger    1520
Name: label, dtype: int64

In [ ]:
df_test_anger

,id,text,label,intensity
0,10941,At the point today where if someone says somet...,anger,0.0
1,10942,CorningFootball IT S GAME DAY T MIN...,anger,0.0
2,10943,This game has pissed me off more than any othe...,anger,0.0
3,10944,spamvicious Ihave just found out itis Candice...,anger,0.0
4,10945,moocowward mrsajhargreaves Melly GaryBar...,anger,0.0
...,...,...,...,...
1515,11696,ggreenwald What if the supposed animosity is ...,anger,0.0
1516,11697,Will BYUis offense score vs WVU,anger,0.0
1517,11698,Id love c Gyimah in action but his coach is ...,anger,0.0
1518,11699,Forgiving means operating with Godis spirit a...,anger,0.0


In [ ]:
express = [fear_test,fear_test_intensity]
df_test_fear = pd.concat(express)
df_test_fear.reset_index(inplace=True)
df_test_fear.label.value_counts()

fear    1990
Name: label, dtype: int64

In [ ]:
express = [sad_test,sad_test_intensity]
df_test_sad = pd.concat(express)
df_test_sad.reset_index(inplace=True)
df_test_sad.label.value_counts()

joy        714
sadness    673
Name: label, dtype: int64

In [ ]:
express = [joy_test,joy_test_intensity]
df_test_joy = pd.concat(express)
df_test_joy.reset_index(inplace=True)
df_test_joy.label.value_counts()

joy    1428
Name: label, dtype: int64

In [ ]:
# X_anger_test = df_test_anger.drop(['intensity'],axis=1)
# Y_anger_actual = df_test_anger['intensity']

In [ ]:
#ANGER
X_anger_test = anger_vectorizer.transform(df_test_anger['text'])
Y_anger_actual = df_test_anger['intensity']
Y_anger_predicted = anger_model.predict(X_anger_test)

#FEAR
X_fear_test = fear_vectorizer.transform(df_test_fear['text'])
Y_fear_actual = df_test_fear['intensity']
Y_fear_predicted = fear_model.predict(X_fear_test)

#SADNESS
X_sad_test = sad_vectorizer.transform(df_test_sad['text'])
Y_sad_actual = df_test_sad['intensity']
Y_sad_predicted = sad_model.predict(X_sad_test)

#JOY
X_joy_test = joy_vectorizer.transform(df_test_joy['text'])
Y_joy_actual = df_test_joy['intensity']
Y_joy_predicted = joy_model.predict(X_joy_test)

In [ ]:
df_test_anger['text'].shape

(1520,)

In [ ]:
get_score(Y_anger_actual, Y_anger_predicted, "Result of Anger Model")
get_score(Y_fear_actual, Y_fear_predicted, "Result of Fear Model")
get_score(Y_sad_actual, Y_sad_predicted, "Result of Sadness Model")
get_score(Y_joy_actual, Y_joy_predicted, "Result of Joy Model")

Result of Anger Model

Mean Absolute Error: 0.12365844642919147
Mean Squared Error: 0.01846753169996347
Root Mean Squared Error: 0.13589529682797513
Result of Fear Model

Mean Absolute Error: 0.3124654070424223
Mean Squared Error: 0.1381831432253937
Root Mean Squared Error: 0.371729933184555
Result of Sadness Model

Mean Absolute Error: 0.30271851262504956
Mean Squared Error: 0.12311940006952747
Root Mean Squared Error: 0.35088374152919577
Result of Joy Model

Mean Absolute Error: 0.316443449738164
Mean Squared Error: 0.1364911543536596
Root Mean Squared Error: 0.36944709276655513


(0.316443449738164, 0.1364911543536596, 0.36944709276655513)

In [ ]:
import scipy

In [ ]:
def evaluate(pred,gold):
    gold_scores=[]
    pred_scores=[]
    gold_scores_range_05_1=[]
    pred_scores_range_05_1=[]
    for p in pred:
        pred_scores.append(p)
    for g in gold:
        gold_scores.append(g)
    for i in range(len(gold_scores)):
        if gold_scores[i] >= 0.5:
            gold_scores_range_05_1.append(gold_scores[i])
            pred_scores_range_05_1.append(pred_scores[i])

    if np.std(pred_scores)==0 or np.std(gold_scores)==0:
        return (0,0,0,0)

    pears_corr=scipy.stats.pearsonr(pred_scores,gold_scores)[0]
    pears_corr_range_05_1=scipy.stats.pearsonr(pred_scores_range_05_1,gold_scores_range_05_1)[0]

    return (pears_corr,pears_corr_range_05_1)

In [ ]:
pear_results=[]
spear_results=[]
pear_results_range_05_1=[]
spear_results_range_05_1=[]

num_pairs = 4
argv = ["Anger_Actual", Y_anger_actual, "Anger_Predicted", Y_anger_predicted, "Fear_Actual", Y_fear_actual, "Fear_Predicted", Y_fear_predicted, "Sad_Actual", Y_sad_actual, "Sad_Predicted", Y_sad_predicted, "Joy_Actual", Y_joy_actual, "Joy_Predicted", Y_joy_predicted]

for i in range(0,len(argv),4):
    name_gold = argv[i]
    gold=argv[i+1]
    name_pred = argv[i+2]
    pred=argv[i+3]
    result=evaluate(pred,gold)
    print ("Pearson correlation between ", name_pred, " and ", name_gold, ":\t", str(result[0]))
    pear_results.append(result[0])
    print ("Pearson correlation for gold scores in range 0.5-1 between ",name_pred," and ",name_gold,":\t",str(result[1]))
    pear_results_range_05_1.append(result[1])

avg_pear=np.mean(pear_results)

avg_pear_range_05_1=np.mean(pear_results_range_05_1)

print ("Average Pearson correlation:\t",str(avg_pear))

print ("Average Pearson correlation for gold scores in range 0.5-1:\t", str(avg_pear_range_05_1))

Pearson correlation between  Anger_Predicted  and  Anger_Actual :	 0
Pearson correlation for gold scores in range 0.5-1 between  Anger_Predicted  and  Anger_Actual :	 0
Pearson correlation between  Fear_Predicted  and  Fear_Actual :	 0.18564979466709902
Pearson correlation for gold scores in range 0.5-1 between  Fear_Predicted  and  Fear_Actual :	 0.44959089209444586
Pearson correlation between  Sad_Predicted  and  Sad_Actual :	 0.45816155345275206
Pearson correlation for gold scores in range 0.5-1 between  Sad_Predicted  and  Sad_Actual :	 0.4210432978823117
Pearson correlation between  Joy_Predicted  and  Joy_Actual :	 0.18755002061283035
Pearson correlation for gold scores in range 0.5-1 between  Joy_Predicted  and  Joy_Actual :	 0.31388824299174056
Average Pearson correlation:	 0.20784034218317035
Average Pearson correlation for gold scores in range 0.5-1:	 0.29613060824212456
